# Онлайн-маркетплейс «Заберу» 

На данной платформе можно приобрести товары ручной работы от небольших производителей. «Заберу» проводит рекламные акции длительностью 1 день.

**Задача:** Узнать на сколько эффективны такие рекламные кампании.

#### Для этого:
- Нужно посчитать затраты на каждый рекламный источник;
- Нужно посчить суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег;
- Ввести новую метрику эффективности рекламы — дневные затраты на рекламу на пользователя по источнику. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника.

## Описание данных

Таблица **visits** (лог сервера с информацией о посещениях сайта):
- uid — уникальный идентификатор пользователя
- device — категория устройства пользователя
- startts — дата и время начала сессии
- endts — дата и время окончания сессии
- sourceid — идентификатор рекламного источника, из которого пришёл пользователь


Таблица **orders** (информация о заказах):
- uid — уникальный идентификатор пользователя, который сделал заказ
- buyts — дата и время заказа
- revenue — выручка Яндекс.Афиши с этого заказа


Таблица **advertisment_costs** (информация о затратах на маркетинг):
- sourceid — идентификатор рекламного источника
- dt — дата
- costs — затраты на этот рекламный источник в этот день


## Оглавление

#### Глава 1. Загрузка библиотек и данных

- Импорт библиотек
- Создаём подключение к базе данных
- Выводим таблицы, смотрим на значения в данных

#### Глава 2. Работа с данными

- Рассчитываем затраты на каждый рекламный источник
- Вывод
- Рассчитываем суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег
- Вывод
- Рассчитываем дневные затраты на рекламу на пользователя по источнику. Вводим новую метрику эффективности рекламы

#### Финальный вывод

### Глава 1. Загрузка библиотек и данных


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine


**Создаём подключение к базе данных**

Для этого создаём конекцию к базе указывая необходимые параметры. Коннекция к базе хранится в переменной engine

In [2]:
db_config = {'user':'praktikum_student',         # имя пользователя
             'pwd':'Sdf4$2;d-d30pp',          # пароль
             'host':'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # порт подключения
             'db':'data-analyst-final-project-db'}          # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string)

### Выводим таблицы, смотрим на значения в данных

In [3]:
query = '''
SELECT *
FROM 
    visits
LIMIT 5    
'''
pd.io.sql.read_sql(query, con = engine)

,id,uid,device,endts,sourceid,startts
0,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00
1,1,104060357244891740,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00
2,2,7459035603376831527,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00
3,3,16174680259334210214,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00
4,4,9969694820036681168,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00


Пользователи используют платформу с мобильной и десктопной версии. Встречаются данные за 2017 год. Пользователи приходят из разных рекламных источников.

In [4]:
query = '''
SELECT *
FROM 
    orders
LIMIT 5    
'''
pd.io.sql.read_sql(query, con = engine)

,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844
3,4,2017-06-01 00:29:00,$0.55,16109239769442553005
4,5,2017-06-01 07:58:00,$0.37,14200605875248379450


Бывает довольно минимальная выручка Яндекс.Афиши с заказа, например 0,37 $

In [5]:
query = '''
SELECT *
FROM 
    advertisment_costs
LIMIT 5    
'''
pd.io.sql.read_sql(query, con = engine)

,id,sourceid,dt,costs
0,1,1,2017-06-01,$75.20
1,2,1,2017-06-02,$62.25
2,3,1,2017-06-03,$36.53
3,4,1,2017-06-04,$55.00
4,5,1,2017-06-05,$57.08


## Глава 2. Работа с данными

### Рассчитываем затраты на каждый рекламный источник

In [6]:
query = '''
SELECT
    sourceid,
    sum(costs) as sum
FROM 
    advertisment_costs
GROUP BY
    sourceid
'''
pd.io.sql.read_sql(query, con = engine)

,sourceid,sum
0,10,"$5,822.49"
1,9,"$5,517.49"
2,1,"$20,833.27"
3,5,"$51,757.10"
4,4,"$61,073.60"
5,2,"$42,806.04"
6,3,"$141,321.63"


Всего 10 рекламных источников. Три из них отсутствуют. Возможно, это связано с технической ошибкой или отсутствием переходов с данных рекламных размещений. 

### Вывод

Самые большие суммарные затраты — 141 тыс., связаны с рекламным размещением **№3**. Далее идёт рекламный источник **№4** — 61 тыс. Примерно в равной степени самые минимальные затраты приходятся на рекламные размещения **№9 и 10** — 5,5 тыс.

### Рассчитываем суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег

In [7]:
query = '''
SELECT sum(revenue) AS sum,
       max(revenue) AS max_revenue,
       min(revenue) AS min_revenue
       
FROM orders o

WHERE 
    o.uid IN (
              SELECT DISTINCT v.uid
              
              FROM visits v
              
              WHERE
                   v.device = 'desktop' AND
                   v.sourceid = '3');
'''
pd.io.sql.read_sql(query, con = engine)

,sum,max_revenue,min_revenue
0,"$106,058.86","$2,633.28",$0.00


Суммарная выручка составляет 106,058 тыс., максимальная — 2,633 $. 
Минимальная выручка равна 0. Возможно, это заказы пользователей, которые приобрели товары бесплатно (акция). 

### Вывод

В целом затраты на рекламный источник № 3 не окупаются с декстопной версии платформы. Разница затрат равна 35,263 тыс.  Конверсия выручки от затрат составляет примерно 75%. Несмотря на то, что большая часть вложений приходится на рекламное размещение № 3, на данный момент оно не приносит прибыли.

Возможно, дополнительная выручка закрывающая часть затрат связанных с рекламным размещением № 3 приходит из мобильной версии платформы. Поэтому, данному рекламному размещению принадлежит большая часть вложений бюджета маркетинга. 

### Рассчитываем дневные затраты на рекламу на пользователя по источнику. Вводим новую метрику эффективности рекламы

In [8]:
query = ''' SELECT
                        new_visits.date_revenue AS date,
                        new_visits.device,
                        new_visits.sourceid,
                        SUM(new_visits.revenue) AS total_revenue,
                        SUM(new_advertisment_costs.costs) AS total_costs,
                        SUM(new_visits.revenue) - SUM(new_advertisment_costs.costs) AS markt_metric
                    FROM (
                        SELECT
                           device,
                           sourceid,
                           SUM(o.revenue) as revenue, 
                           DATE_TRUNC('day', o.buyts)::date AS date_revenue
                        FROM
                           orders o
                        RIGHT JOIN (SELECT 
                                        uid, 
                                        MAX(sourceid) AS sourceid 
                                    FROM
                                        visits 
                                    GROUP BY 
                                        uid
                                    ) AS table_visits_1 ON table_visits_1.uid = o.uid
                        INNER JOIN (SELECT 
                                        uid, 
                                        MAX(device) AS device 
                                    FROM 
                                        visits 
                                    GROUP BY 
                                        uid
                                    ) AS table_visits_2 ON table_visits_2.uid = o.uid
                        GROUP BY 
                            table_visits_2.device, table_visits_1.sourceid, date_revenue
                                   ) AS new_visits
                    INNER JOIN (SELECT
                                    sourceid, 
                                    SUM(costs) AS costs, 
                                    advertisment_costs.dt::date AS date_costs 
                                FROM 
                                    advertisment_costs
                                GROUP BY 
                                    sourceid, 
                                    date_costs
                               ) AS new_advertisment_costs 
                                 ON new_visits.sourceid = new_advertisment_costs.sourceid 
                                 AND new_visits.date_revenue = new_advertisment_costs.date_costs
 
                    GROUP BY
                        new_visits.device, 
                        new_visits.sourceid,
                        new_visits.date_revenue
                    ORDER BY
                        date, sourceid
                '''
 
pd.io.sql.read_sql(query, con = engine)

,date,device,sourceid,total_revenue,total_costs,markt_metric
0,2017-06-01,desktop,1,$139.36,$75.20,$64.16
1,2017-06-01,desktop,2,$3.79,$132.56,-$128.77
2,2017-06-01,touch,2,$11.55,$132.56,-$121.01
3,2017-06-01,desktop,3,$71.69,$349.38,-$277.69
4,2017-06-01,touch,3,$10.46,$349.38,-$338.92
...,...,...,...,...,...,...
4656,2018-05-31,touch,5,$61.11,$398.04,-$336.93
4657,2018-05-31,desktop,9,$65.51,$27.37,$38.14
4658,2018-05-31,touch,9,$75.35,$27.37,$47.98
4659,2018-05-31,desktop,10,$309.98,$17.60,$292.38


Судя по таблице, второй и третий рекламный источник по мобильной и десктопной версии не окупаются. Затраты превышают выручку. Тоже самое касается рекламного размещения №5 по мобилке. 

Рекламные размещения №9 и №10 — окупаются. Видимо, лучше всего отрабатывает рекламный источник №10 по десктопной версии. 




UPD: Думаю я не внимательно посмотрел условия третьей задачи.

Метрика отображает отношение рекламных затрат по данному рекламному источнику к количеству посетителей, перешедших на сайт с данного источника. 

In [9]:
query ='''   SELECT
                    t.date,
                    t.sourceid,
                    advertisment_costs.costs / t.cnt_users as metrica
            FROM
                    advertisment_costs JOIN
                            (
                                SELECT
                                    DATE_TRUNC('day', startts::timestamp) as date,
                                    count(uid) as cnt_users,
                                    sourceid
                                FROM
                                    visits
                                GROUP BY
                                    sourceid,
                                    date
        ) as t ON advertisment_costs.sourceid = t.sourceid AND 
        DATE_TRUNC('day', advertisment_costs.dt::timestamp) = t.date

ORDER BY 
    t.date, 
    t.sourceid'''

pd.io.sql.read_sql(query, con = engine)

,date,sourceid,metrica
0,2017-06-01,1,$1.10
1,2017-06-01,2,$1.55
2,2017-06-01,3,$1.88
3,2017-06-01,4,$0.54
4,2017-06-01,5,$0.60
...,...,...,...
2536,2018-05-31,3,$2.03
2537,2018-05-31,4,$0.96
2538,2018-05-31,5,$1.57
2539,2018-05-31,9,$0.44


## Финальный вывод

Мы проанализировали стратегию рекламной кампании маркетплейса «Заберу». Она выстраивается на рекламных акциях длительностью 1 день. В целом рекламная кампания показала свою неэффективность. 

Самые большие суммарные затраты — 141 тыс., связаны с рекламным размещением №3. Далее идёт рекламный источник №4 — 61 тыс. Примерно в равной степени самые минимальные затраты приходятся на рекламные размещения №9 и 10 — 5,5 тыс. Основная часть рекламных размещений не окупается. Видимо, лучше всего отрабатывает рекламный источник №10 по десктопной версии.